In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

# Classificação de texto

In [2]:
#Verifica os dados no diretorio

Data = os.listdir("../../Filter/Tweets_Filtered/")
for i in range(len(Data)):
    Data[i] = Data[i].split(".")[0]
    
print(Data)

['cartacapital', 'conexaopolitica', 'EstadaoEconomia', 'EstadaoPolitica', 'folha', 'folha_mercado', 'folha_poder', 'g1', 'g1economia', 'g1politica', 'GloboNews', 'hbredda', 'jairbolsonaro', 'OGloboPolitica', 'OGlobo_Economia', 'pedrocerize', 'RevistaEpoca', 'RevistaISTOE', 'UOL', 'UOLEconomia', 'valoreconomico', 'VEJA']


In [3]:
Pessoas = ["realDonaldTrump", "jairbolsonaro", "hbredda", "gui_benchimol", "pedrocerize"]

NoticiasPolitica = ["OGloboPolitica", "folha_poder", "GloboNews", "EstadaoPolitica", 
                    "RevistaEpoca", "valoreconomico", "g1politica", "conexaopolitica", "EstadaoEconomia", 
                    "UOLEconomia", "folha_mercado", "g1economia", "OGlobo_Economia", "valoreconomico"]

Noticias = ["UOL", "folha", "g1", "VEJA", "cartacapital", "RevistaISTOE"]

In [4]:
#Monta um dicionario com todos datasets de twites de todas as pessoas
Alvo = NoticiasPolitica


AllData = {}

for i in Alvo:
    if i in Data:
        AllData[i] = pd.read_csv("../../Filter/Tweets_Filtered/{0}".format(i+".csv"), sep=";", decimal=",", encoding='utf-16')
    else:
        print("{0} não disponivel".format(i))

In [5]:
printrange = range(0, 5)
print("-"*10)
for i in printrange:
    print(AllData[Alvo[0]]["Texto"].iloc[i])
    print("-"*10)

----------
carlos ignora ponto final do pai e volta a atacar mourão estranhíssimo alinhamento com políticos que detestam o… 
----------
polícia civil deflagra operação em todo o país contra foragidos da justiça 
----------
análise decisão do stj sobre lula não muda situação do pt 
----------
bom dia leitor siga aqui as principais notícias da política nacional e os acontecimentos mais importantes pelo pa… 
----------
lula pode ser solto após decisão do stj entenda 
----------


In [6]:
AllX = {}

for i in Alvo:
    AllX[i] = list(AllData[i]["Texto"])

In [7]:
print(AllX[Alvo[0]][:2])

['carlos ignora ponto final do pai e volta a atacar mourão estranhíssimo alinhamento com políticos que detestam o… ', 'polícia civil deflagra operação em todo o país contra foragidos da justiça ']


In [8]:
X = []

for i in Alvo:
    X = X + AllX[i]

## Filtra os dataframes para agrupar os tweets

In [9]:
Test = AllData[Alvo[0]].head(10)
Test

,Pessoa,Data,Texto,Retweets,Likes,URL
0,OGloboPolitica,2019-04-24 12:56:33,carlos ignora ponto final do pai e volta a ata...,8,30,1
1,OGloboPolitica,2019-04-24 10:58:35,polícia civil deflagra operação em todo o país...,2,13,1
2,OGloboPolitica,2019-04-24 10:35:24,análise decisão do stj sobre lula não muda sit...,1,6,1
3,OGloboPolitica,2019-04-24 10:21:14,bom dia leitor siga aqui as principais notícia...,0,1,1
4,OGloboPolitica,2019-04-24 00:04:58,lula pode ser solto após decisão do stj entenda,0,13,1
5,OGloboPolitica,2019-04-23 22:59:07,advogado de lula promete apresentar todos os r...,2,22,1
6,OGloboPolitica,2019-04-23 22:49:05,funcionário da vale diz que diretores sabiam d...,5,17,1
7,OGloboPolitica,2019-04-23 22:40:49,bolsonaro quer ponto final em briga entre carl...,3,12,1
8,OGloboPolitica,2019-04-23 22:22:42,“não esperava nada positivo sobre o julgamento...,12,46,1
9,OGloboPolitica,2019-04-23 22:04:21,quando um não quer dois não brigam diz mourão...,35,181,1


In [10]:
import datetime

DateInterval = [datetime.datetime.strptime("17:00:00", "%H:%M:%S").time(), 
                datetime.datetime.strptime("9:00:00", "%H:%M:%S").time()]

oneday = datetime.timedelta(days=1)
daytweets = []
tweets_data = {}

for i in Alvo:
    datas = AllData[i]['Data']
    tweets = AllData[i]['Texto']
    likes = AllData[i]['Likes']
    retweets = AllData[i]['Retweets']
    Started = True
    lastday = datetime.datetime.strptime(datas[0], '%Y-%m-%d %H:%M:%S').date()
    
    for j in range (0, len(datas)):
        
        data = datetime.datetime.strptime(datas[j], '%Y-%m-%d %H:%M:%S').date()
        time = datetime.datetime.strptime(datas[j], '%Y-%m-%d %H:%M:%S').time()
        
        if data != lastday:
            delta = abs(int(str(data - lastday).split(" ")[0]))
            if int(delta) >= 2:
                #Acaba o periodo por força bruta
                Started = False
                if(str(data) not in tweets_data):
                    tweets_data[str(data)] = daytweets
                else:
                    for k in daytweets:
                        tweets_data[str(data+oneday)].append(k)
                daytweets = []
        
        #Test to 00:00
        First = (DateInterval[0] < time) and (time < datetime.datetime.strptime("23:59:59", "%H:%M:%S").time())
        #Test from 00:00
        Second = (datetime.datetime.strptime("00:00:01", "%H:%M:%S").time() < time) and (time < DateInterval[1])

        if First or Second:
            Started = True
            #Esta dentro do periodo
            daytweets.append(tweets[j])

        elif Started:
            #Acaba o periodo
            Started = False
            if(str(data+oneday) not in tweets_data):
                tweets_data[str(data+oneday)] = daytweets
            else:
                for k in daytweets:
                    tweets_data[str(data+oneday)].append(k)
            daytweets = []
                
        lastday = data

Dates = list(tweets_data.keys())

for i in range(3):
    print(Dates[i], tweets_data[Dates[i]][0:2])

2019-04-25 ['rt expressoepoca desistência de márcio lacerda agita eleições em minas gerais ex prefeito de belo horizonte abriu mão da candidatura nest…', 'rt expressoepoca stj solta executivo de multinacional da área da saúde preso na lava jato trata se de antonio georgete ']
2019-04-24 ['lula pode ser solto após decisão do stj entenda ', 'advogado de lula promete apresentar todos os recursos que lei permite para absolvê lo ']
2019-04-23 ['dono oficial do sítio de atibaia pede autorização judicial para vendê lo ', 'defesa de lula diz que não foi intimada sobre julgamento no stj pautado para terça ']


## Filtra as ações

In [ ]:
bolsa = pd.read_excel('../../Dados_Bolsa/Twt.xlsx', sheet_name="1h", usecols=[0,1,2,3])

In [12]:
bolsa.head(5)

,ibov,Price,Volume,Ticks
0,2018-10-08 10:00:00,86283.67,0,119
1,2018-10-08 11:00:00,85686.68,0,120
2,2018-10-08 12:00:00,85408.61,0,120
3,2018-10-08 13:00:00,85267.66,0,120
4,2018-10-08 14:00:00,85686.73,0,120


In [13]:
ibov_len  = len(bolsa['ibov'])
ibov_len

1110

## Separa os resultados da IBov em classes (zonas)

In [ ]:
print(len(x[0]), len(y))

In [ ]:
print("Numero de 0s:", y.count(0))
print("Numero de 1s:", y.count(1))

In [ ]:
print("Proporção de 0s:", y.count(0)/len(y))
print("Proporção de 1s:", y.count(1)/len(y))

In [16]:
from sklearn.model_selection import cross_val_predict, cross_val_score

In [19]:
x=pd.DataFrame(X)

In [20]:
ibov_data_precos_v2 = {}
# célula para identificar os preços de abertura e fechamento de cada dia
for i in range(0, ibov_len):
    data_completa = datetime.datetime.strptime(str(bolsa['ibov'][i]), '%Y-%m-%d %H:%M:%S')
    preco = bolsa['Price'][i]
    data = data_completa.strftime("%Y-%m-%d")
    if data not in ibov_data_precos_v2:
        ibov_data_precos_v2[data] = {}
        ibov_data_precos_v2[data]['abertura'] = preco
        ibov_data_precos_v2[data]['fechamento'] = preco #caso só tenhamos informação de uma hora dentro de um dia, assumimos esse preco como o de fechamento tbm
        dia_anterior_completo =  data_completa - datetime.timedelta(days=1)
        dia_anterior_data = dia_anterior_completo.strftime("%Y-%m-%d")
        if(dia_anterior_data in ibov_data_precos_v2):
            dia_anterior_fechamento = ibov_data_precos_v2[dia_anterior_data]['fechamento']
            delta_fechamento=preco/dia_anterior_fechamento-1
            if(delta_fechamento>(0.2*10**-2)):
                ibov_data_precos_v2[data]['variacao'] = 2
            elif(delta_fechamento<(-0.2*10**-2)):
                ibov_data_precos_v2[data]['variacao'] = 0
            else:
                ibov_data_precos_v2[data]['variacao'] = 1
        else:
            ibov_data_precos_v2[data]['variacao'] = 0 #se não tivermos informações sobre o dia anterior, assumimos que a variacao caiu
    else:
        if i < (ibov_len - 1):
            data_seguinte = datetime.datetime.strptime(str(bolsa['ibov'][i+1]), '%Y-%m-%d %H:%M:%S')
            if data_seguinte.date() > data_completa.date():
                ibov_data_precos_v2[data]['fechamento'] = preco
                
Datesibov = list(ibov_data_precos_v2.keys())
                
for i in range(4):
    print(Datesibov[i], ibov_data_precos_v2[Datesibov[i]])

2018-10-08 {'abertura': 86283.67, 'fechamento': 86083.91, 'variacao': 0}
2018-10-09 {'abertura': 85864.42, 'fechamento': 86087.55, 'variacao': 0}
2018-10-10 {'abertura': 84590.04, 'fechamento': 83679.11, 'variacao': 0}
2018-10-11 {'abertura': 84492.8, 'fechamento': 82921.08, 'variacao': 2}


In [43]:
len(X)

44665

In [44]:
YBits = []
XBits = []

for data in ibov_data_precos_v2:
    if(data in tweets_data):
        for j in tweets_data[data]:
            XBits.append(j)
            YBits.append(ibov_data_precos_v2[data]['variacao'])

KeyError: '2018-10-29'

In [42]:
from tensorflow.keras.utils import to_categorical

classes=to_categorical(class_list)
print(classes)

classest = np.transpose(classes)

for i in range(len(classest)):
    print(list(classest[i]).count(1)/len(classest[i]))

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
0.442419623059867
0.17281042128603105
0.384769955654102


A separação por zonas, criou uma classe com 44% de predominância, ou seja, temos uma baseline para um modelom classificador de 44%

## Geração de sequência a partir dos textos em níveis de carácteres

In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(XBits)

Quando o Tokenizer é inicializado apenas dois parâmetros são importantes.
char_level=True: isso torna a ação texts_to_sequences() pronta para processar o texto em nível de carácter
oov_token='UNK': isso adicionar um carácter desconhecido do vacubulário, o que permite que mesmo se novos caracteres aparecerem o código será capaz de rodar

In [23]:
tk.word_index

{'UNK': 1,
 ' ': 2,
 'a': 3,
 'e': 4,
 'o': 5,
 'r': 6,
 'i': 7,
 's': 8,
 'd': 9,
 'n': 10,
 't': 11,
 'm': 12,
 'c': 13,
 'l': 14,
 'p': 15,
 'u': 16,
 'v': 17,
 'b': 18,
 'g': 19,
 'f': 20,
 'ã': 21,
 'h': 22,
 'q': 23,
 'ç': 24,
 'z': 25,
 'j': 26,
 'á': 27,
 'é': 28,
 'x': 29,
 'í': 30,
 '…': 31,
 'ê': 32,
 '1': 33,
 'ó': 34,
 '0': 35,
 '2': 36,
 'õ': 37,
 'ú': 38,
 '3': 39,
 'w': 40,
 'à': 41,
 'â': 42,
 '8': 43,
 'y': 44,
 '5': 45,
 '9': 46,
 '4': 47,
 'k': 48,
 '7': 49,
 '6': 50,
 'ô': 51,
 '‘': 52,
 '’': 53,
 '“': 54,
 'º': 55,
 '”': 56,
 'ª': 57,
 '—': 58,
 '👇': 59,
 '\u2066': 60,
 '–': 61,
 'ü': 62,
 '\u2069': 63,
 '️': 64,
 '😍': 65,
 '\xa0': 66,
 '°': 67,
 '👀': 68,
 '❤': 69,
 '🎉': 70,
 '😉': 71,
 '🤔': 72,
 '´': 73,
 '😱': 74,
 '🏻': 75,
 '🤩': 76,
 '👏': 77,
 '🎶': 78,
 '👍': 79,
 '😂': 80,
 '✈': 81,
 '🇸': 82,
 '🇦': 83,
 '🤣': 84,
 'ë': 85,
 '🎬': 86,
 '💪': 87,
 '🇧': 88,
 '🇷': 89,
 '🌊': 90,
 '👆': 91,
 '⚡': 92,
 'ñ': 93,
 '🍻': 94,
 '🥚': 95,
 '🍳': 96,
 '👫': 97,
 '🏼': 98,
 '🐣': 99,
 '🧬'

In [24]:
sequences = tk.texts_to_sequences(XBits)

In [25]:
for i, s in enumerate(sequences):
    print(len(s))
    if i > 4:
        break

60
115
117
114
73
88


In [26]:
input_size=140

pad_X = pad_sequences(sequences, maxlen=input_size, padding='post')

In [27]:
pad_X=np.array(pad_X)
pad_X.shape

(14432, 140)

In [30]:
vocab_size=len(tk.word_index)
vocab_size

122

In [31]:
embedding_weights=[]
embedding_weights.append(np.zeros(vocab_size))

for char, i in tk.word_index.items():
    onehot=np.zeros(vocab_size)
    onehot[i-1]=1
    embedding_weights.append(onehot)
embedding_weights=np.array(embedding_weights)


In [32]:
print(embedding_weights.shape)
embedding_weights

(123, 122)


array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

## Montando a rede neural

In [33]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.layers import LSTM, Dropout
from tensorflow.keras.models import Model

num_of_classes=3
input_size=input_size
embedding_size=vocab_size
dropout_p = 0.5
optimizer = 'adam'
loss = 'categorical_crossentropy'


embedding_layer=Embedding(vocab_size +1, 
                         embedding_size,
                         input_length=input_size)

In [34]:
print(pad_X.shape)
print(classes.shape)

(14432, 140)
(14432, 3)


In [35]:
# Model construction
inputs = Input(shape=(input_size,))
embedded_sequence = embedding_layer(inputs)
x = LSTM(256, return_sequences=True, activation='selu')(embedded_sequence)
x = Flatten()(x)
x = Dense(512, activation='selu')(x)
x = Dropout(dropout_p)(x)
prediction = Dense(num_of_classes, activation='softmax')(x)

model = Model(inputs=inputs, outputs=prediction)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 140)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 140, 122)          15006     
_________________________________________________________________
unified_lstm (UnifiedLSTM)   (None, 140, 256)          388096    
_________________________________________________________________
flatten (Flatten)            (None, 35840)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               18350592  
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 1539  

In [41]:
from sklearn.model_selection import train_test_split

SEED=42
x_train, x_test, y_train, y_test = train_test_split(pad_X, classes, test_size=.07, random_state=SEED)

In [101]:
# training
model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=64,
          epochs=5,
          verbose=1)

Train on 13421 samples, validate on 1011 samples
Epoch 1/5
13421/13421 [==============================] - 268s 20ms/sample - loss: 1.0291 - accuracy: 0.4679 - val_loss: 1.0269 - val_accuracy: 0.4599
Epoch 2/5
13421/13421 [==============================] - 275s 20ms/sample - loss: 0.9953 - accuracy: 0.4968 - val_loss: 1.0353 - val_accuracy: 0.4491
Epoch 3/5
13421/13421 [==============================] - 263s 20ms/sample - loss: 0.9671 - accuracy: 0.5159 - val_loss: 1.0401 - val_accuracy: 0.4777
Epoch 4/5
13421/13421 [==============================] - 261s 19ms/sample - loss: 0.9431 - accuracy: 0.5452 - val_loss: 1.0598 - val_accuracy: 0.4441
Epoch 5/5
13421/13421 [==============================] - 316s 24ms/sample - loss: 0.9218 - accuracy: 0.5608 - val_loss: 1.0611 - val_accuracy: 0.4649


In [102]:
model.save('hope.h5')

In [ ]:
from tensorflow.keras.models import load_model
# load model from single file
model = load_model('hope.h5')

In [ ]:
score, acc = model.evaluate(x_test, y_test)